
### Examining racial discrimination in the US job market

#### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

#### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes.

#### Exercise
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Discuss statistical significance.

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

****

In [42]:
import pandas as pd
import numpy as np
from scipy import stats
import math

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [19]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [10]:
data.columns
data.head(2)

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,


#### What test is appropriate for this problem? Does CLT apply?

Let's being by looking the number of observations where race = b and race = w. We can also check the number of such observations that have received a call back and the ones that didn't. Using these factors, we will be able to create a contingency table. 

Hence, the problem boils down to comparison of two proportions. 
If certain conditions are satisfied, we can also perform the Fischer's Exact Test using the contingency table.


In [73]:
data_b = data[data.race=='b']
data_w = data[data.race=='w']
num_b = len(data_b)
num_w = len(data_w)
print "Number of observations where race is b : ",num_b
print "Number of observations where race is w : ",num_w

Number of observations where race is b :  2435
Number of observations where race is w :  2435


Since there can be only two states for the 'call' variable, we can arbitrarily assign getting a call back as "success" and not getting a call back as a failure. 

In [74]:
b_success = len(data_b[data_b.call == 1])
w_success = len(data_w[data_w.call == 1])
print b_success, w_success

157 235


Let's also calculate the proportion $\hat{p}_b$ of black sounding names getting a callback and the proportion $\hat{p}_w$ white sounding names getting a call back.

In [75]:
p_b = 1.0 * b_success/num_b
p_w = 1.0 * w_success/num_w
print "Proportion of black sounding names getting a callback : ",p_b
print "Proportion of white sounding names getting a callback : ",p_w

Proportion of black sounding names getting a callback :  0.064476386037
Proportion of white sounding names getting a callback :  0.0965092402464


Also, to make sure the samples are big enough such that we can use a normal distribution to model difference between 
proportions, we need to check if $n*p$ and $n*(1-p)$ are greater than 10. This is a conclusion from the Central Limit Theorem. 

In [76]:
print num_b * p_b
print num_b * (1-p_b)
print "---"
print num_b * p_w
print num_b * (1-p_w)

157.0
2278.0
---
235.0
2200.0


Since, all the values are above 10, we can use the normal distribution to model differences between proportions.

Now that we have all the required data, we formulate the null and alternate hypotheses.

$H_0\:is \: p_b = p_w\\
H_A \:is \: p_b \neq p_w$

The Standard Error for the sample statistic is given by
$\sqrt{\frac{\hat{p}_b(1-\hat{p}_b)}{n_b} + \frac{\hat{p}_w(1-\hat{p}_w)}{n_w}} $

We can use the z-statistic to place a confidence interval on this sample statistic.Hence, the margin of error is 
$Z_{\alpha/2} * SE$. For a 95% confidence interval, the z-value is 1.96. 

The confidence interval, subsequently, is $\hat{p}_b - \hat{p}_w \pm {Z_{\alpha/2} * SE}$

In [77]:
z = 1.96
margin = z * math.sqrt( ( p_w*(1-p_w) / num_b) + (p_b*(1-p_b)/num_w) )

print "Margin of error = ", margin

Margin of error =  0.0152554063499


In [78]:
print "The confidence interval is given by :", p_w-p_b-z*margin,"to", p_w-p_b+z*margin

The confidence interval is given by : 0.00213225776367 to 0.0619334506552


0 is not in this confidence interval. Nevertheless, let's go ahead and calculating the p-value.

In [79]:
from statsmodels.stats.proportion import proportions_ztest as pz
pz(np.array([b_success,w_success]),np.array([num_b,num_w]),value=0)

(-4.1084121524343464, 3.9838868375850767e-05)

The second value is the p-value and it is much lesser than 0.05. Hence, we can reject the null hypothesis. 

#### Significance of the result
What does it practically mean to reject the null hypothesis? Our null hypothesis was that the proportion of black sounding names getting a call back is equal to the number of white sounding names getting a call back. After analysis, we have decided to reject it. This means that, in reality, there is a significant difference in the number of call backs ; white sounding names getting more call backs.